In [1]:
# Importando as bibliotecas
from CoolProp.CoolProp import PropsSI, PhaseSI
import pprint
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Calcula qualquer ponto informando dicionario com duas variaveis
def calculate_point(point):
    variables = ['T', 'H', 'S', 'Q', 'P']
    input_variables = list(point.keys())
    output_variables = [variable for variable in variables if variable not in input_variables]
    output_values = PropsSI(output_variables, 
                            input_variables[0], 
                            point[input_variables[0]], 
                            input_variables[1], 
                            point[input_variables[1]], 
                            point['refrigerant']
    )

    for index, variable in enumerate(output_variables):
        point[variable] = output_values[index]

    return point

In [3]:
# Calcula ciclo baseado nas variaveis de entrada
def calculate_cycle(cycle_inputs):
    t_0 = cycle_inputs['t_external_env']

    # Point 3 (after condenser)
    point_3_saturado = {'Q': 0, 
                        'T': cycle_inputs['t_external_env'] + cycle_inputs['approach_condenser'],
                       'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_3_saturado)

    point_3 = {'P': point_3_saturado['P'], 
               'T': point_3_saturado['T'] - cycle_inputs['subcooling'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_3)
    
    # Point 5 (after high temperature evaporator)    
    point_5_saturado = {'Q': 1, 
                        'T': cycle_inputs['t_internal_env_ht'] - cycle_inputs['approach_evaporator_ht'],
                        'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_5_saturado)

    point_5 = {'P': point_5_saturado['P'], 
               'T': point_5_saturado['T'] + cycle_inputs['superheating_ht'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_5)
    
    # Point 8 (after low temperature evaporator)  
    point_8_saturado = {'Q': 1, 
                        'T': cycle_inputs['t_internal_env_lt'] - cycle_inputs['approach_evaporator_lt'],
                        'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_8_saturado)

    point_8 = {'P': point_8_saturado['P'], 
               'T': point_8_saturado['T'] + cycle_inputs['superheating_lt'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_8)
    
    # Point 4 (after expansion valve of HTE)  
    point_4 = {'P': point_5['P'], 
               'H': point_3['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_4)

    # Point 7 (after expansion valve of LTE)  
    point_7 = {'P': point_8['P'], 
               'H': point_3['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_7)
    
    # Point 6 (after regulator expansion valve)  
    point_6 = {'P': point_8['P'], 
               'H': point_5['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_6)
    
    # Point 1 (before compressor)
    point_1 = {'P': point_8['P'],
               'H': (1 - cycle_inputs['f']) * point_6['H'] + cycle_inputs['f'] * point_8['H'],
               'refrigerant': cycle_inputs['refrigerant']
    }
    calculate_point(point_1)
    
    # Point 2 (after compressor)
    point_2_isen = {'S': point_1['S'], 
                    'P': point_3['P'],
                    'refrigerant': cycle_inputs['refrigerant']
                   }
    calculate_point(point_2_isen)

    point_2 = {'P': point_3['P'], 
               'H': point_1['H'] + (point_2_isen['H'] - point_1['H']) / cycle_inputs['isentropic_efficiency'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_2)
    
    # Masses
    m = cycle_inputs['work'] / (point_2['H'] - point_1['H'])
    m_lt = m * cycle_inputs['f']
    m_ht = m * (1 - cycle_inputs['f'])
    
    # COP
    q_evaporator_ht = m_ht * (point_5['H'] - point_4['H'])
    q_evaporator_lt = m_lt * (point_8['H'] - point_7['H'])
    cop = (q_evaporator_ht + q_evaporator_lt) / cycle_inputs['work'] 
    
    return {
        'cycle_inputs': cycle_inputs,
        'point_1': point_1,
        'point_2': point_2,
        'point_3': point_3,
        'point_4': point_4,
        'point_5': point_5,
        'point_6': point_6,
        'point_7': point_7,
        'point_8': point_8,
        'm': m,
        'm_ht': m_ht,
        'm_lt': m_lt,
        'q_evaporator_ht': q_evaporator_ht,
        'q_evaporator_lt': q_evaporator_lt,
        'cop': cop
    }

In [4]:
# Variaveis de entrada
input_values = {
    't_external_env': 32.22 + 273.15,
    't_internal_env_ht': 4.44 + 273.15,
    't_internal_env_lt': -17.78 + 273.15,
    'approach_condenser': 0,
    'approach_evaporator_ht': 0,
    'approach_evaporator_lt': 0,
    'work': 18299.697604861303,
    'f': 0.3387218287321152,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'refrigerant': 'NH3'
}

cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

{'cop': 2.8852935791667793,
 'cycle_inputs': {'approach_condenser': 0,
                  'approach_evaporator_ht': 0,
                  'approach_evaporator_lt': 0,
                  'f': 0.3387218287321152,
                  'isentropic_efficiency': 0.7,
                  'refrigerant': 'NH3',
                  'subcooling': 5,
                  'superheating_ht': 5,
                  'superheating_lt': 5,
                  't_external_env': 305.37,
                  't_internal_env_ht': 277.59,
                  't_internal_env_lt': 255.36999999999998,
                  'work': 18299.697604861303},
 'm': 0.04621471474880705,
 'm_ht': 0.030560782054758068,
 'm_lt': 0.01565393269404898,
 'point_1': {'H': 1616393.3717983814,
             'P': 209507.0301798974,
             'Q': -1.0,
             'S': 6468.565716701363,
             'T': 267.99108690371645,
             'refrigerant': 'NH3'},
 'point_2': {'H': 2012364.60529677,
             'P': 1245482.8712624237,
             'Q': -1

In [5]:
# Variaveis de entrada
input_values = {
    't_external_env': 35 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': -15 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.1,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'refrigerant': 'R134a'
}

cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

{'cop': 2.302935178903305,
 'cycle_inputs': {'approach_condenser': 5,
                  'approach_evaporator_ht': 5,
                  'approach_evaporator_lt': 5,
                  'f': 0.1,
                  'isentropic_efficiency': 0.7,
                  'refrigerant': 'R134a',
                  'subcooling': 5,
                  'superheating_ht': 5,
                  'superheating_lt': 5,
                  't_external_env': 308.15,
                  't_internal_env_ht': 278.15,
                  't_internal_env_lt': 258.15,
                  'work': 900},
 'm': 0.01356147000055496,
 'm_ht': 0.012205323000499464,
 'm_lt': 0.001356147000055496,
 'point_1': {'H': 401826.5407648307,
             'P': 132734.97949922277,
             'Q': -1.0,
             'S': 1799.5266394923467,
             'T': 271.8189645356461,
             'refrigerant': 'R134a'},
 'point_2': {'H': 468191.0277978124,
             'P': 1016593.0221206467,
             'Q': -1.0,
             'S': 1856.5256672604

In [6]:
# Otimizacao do f
 
# Calcula proximo k

def calculate_next(opt_input_values, delta, alpha):
    
    current_cycle = calculate_cycle(opt_input_values)
    current_f = opt_input_values['f']
    
    opt_input_values['f'] += delta
    gradient_cycle = calculate_cycle(opt_input_values)
    gradient_f = (gradient_cycle['cop'] - current_cycle['cop']) / delta
    
    # Next f
    opt_input_values['f'] = current_f + alpha*gradient_f
        
# Otimiza superheating e subcooling pelo cop

def optimize_f_with_cop():
    k = 0
    while k < 100 and opt_input_values['f'] > 0.1 and opt_input_values['f'] < 0.9 :
        next_pass = calculate_next(opt_input_values, 10**(-8), 0.2)
        k += 1
    print_cycle = calculate_cycle(opt_input_values)
    print('#'*30)
    print('f: ', print_cycle['cycle_inputs']['f'])
    print('COP: ', print_cycle['cop'])
    print('#'*30)

# Variaveis de entrada para otimizacao
opt_input_values = {
    't_external_env': 35 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': -15 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'refrigerant': 'R134a'
}
    
optimize_f_with_cop()

##############################
f:  0.09574594754485588
COP:  2.3030760764615965
##############################
